# Project ML / DSA : Sentiment Analysis dengan CV ANN Classifier

### Read Feature (Text) and Target (Rating)

In [2]:
from sklearn.datasets import load_files
import numpy as np

reviews = load_files("dataset", encoding="ISO-8859-1")
texts, rating = reviews.data, reviews.target

### Custom Test-Train Text Split Method

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split as tts

def train_test_text_split(X, y, test_size=0.25):
    # normal split
    text_train, text_test, y_train, y_test = tts(texts, rating, test_size=test_size)
    
    # define tokenizer
    token = RegexpTokenizer(r'[a-zA-Z0-9]+')

    # define the vectorizer
    vect = CountVectorizer(min_df=5, lowercase=True,
                           stop_words='english',
                           ngram_range = (1,1),
                           tokenizer = token.tokenize)
    
    # vectorize text
    X_train = vect.fit(text_train).transform(text_train).todense()
    X_test = vect.transform(text_test).todense()
    
    return X_train, X_test, y_train, y_test
    

### Test Train Split

In [5]:
X_train, X_test, y_train, y_test = train_test_text_split(texts, rating, test_size=0.25)

### Vectorizing Text to get the trainable features

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
from nltk.tokenize import RegexpTokenizer

# define tokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')

# define the vectorizer
vect = CountVectorizer(min_df=5, lowercase=True,
                       stop_words='english',
                       ngram_range = (1,1),
                       tokenizer = token.tokenize)

# define the PCA
# pca = PCA(n_components=50)

# define the scaler
# scaler = StandardScaler()

# transform text into trainable vectors

X_train = vect.fit(text_train).transform(text_train).todense()
X_test = vect.transform(text_test).todense()
# X_train_pca = pca.fit(X_train).transform(X_train)
# X_test_pca = pca.transform(X_test)

### Building The ANN Classifier

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from sklearn.naive_bayes import MultinomialNB
import numpy as np

def create_ann_clf(feature_count, num_classes):
    classifier = Sequential()
    hidden_units = (feature_count + 1) // 2
    classifier.add(Dense(output_dim = hidden_units, init = 'uniform', 
                         activation = 'relu', input_dim = feature_count))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(output_dim = hidden_units, activation = 'relu'))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(output_dim = num_classes, init = 'uniform', 
                         activation = 'softmax'))
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', 
                       metrics = ['accuracy'])
    return classifier


Using TensorFlow backend.


### Create the Hybrind ANN-NB class

In [7]:
class HybridANNBayesClassifier:
    
    def __init__(self):
        self.bayes_model = MultinomialNB()
        
    def get_x_with_dk(self, X):
        domain_knowledge = self.ann_model.predict(X)
        return np.concatenate([np.array(X),np.matrix(np.argmax(domain_knowledge, axis=1)).transpose()], axis=1)
    
    def fit(self, X, y):
        self.ann_model = create_ann_clf(X.shape[1], len(set(y)))
        self.ann_model.fit(X, to_categorical(y))
        self.bayes_model.fit(self.get_x_with_dk(X), y)
        
    def score(self, X, y):
        return self.bayes_model.score(self.get_x_with_dk(X), y)
    
    def predict(self, X, y):
        return self.bayes_model.predict(self.get_x_with_dk(X))        

### Comparison

Beside Baseline (Dummy Classifier) and Hybrid ANN-NB, there are 3 other algorithm that'll be used for comparison. They are Logistic Regression, MLPClassifier (ANN), and Naive Bayes.

#### Performance Report Method

In [35]:
import numpy as np
def report_performance(report):
    for model, scores in report.items():
        print("Model: ", model)
        print("Max: ", np.max(scores))
        print("Min: ", np.min(scores))
        print("Avg: ", np.mean(scores))
        print()

#### Model Performance Measuring Method (complete with cross validation)

In [22]:
def compare_performances(models, X, y, cv=3):
    scores = {}
    
    # initiate scores
    for name in models:
        scores[name] = []
        
    # cross-validate as demanded
    for i in range(cv):
        
        # all models use the same train-test data per cv
        X_train, X_test, y_train, y_test = train_test_text_split(X, y, test_size=0.25)
        
        # iterate all models
        for name, model in models.items():
            model.fit(X_train,y_train)
            scores[name].append(model.score(X_test, y_test))
    return scores

#### Defining Models

Aside from Hybrid ANN-NB model, all other model are used from sklearn library

In [27]:
models = {'Baseline': DummyClassifier(),
          'HybridANN-NB': HybridANNBayesClassifier(),
          'ANN': MLPClassifier(),
          'Naive Bayes': MultinomialNB(),
          'Logistic Regression': LogisticRegression()
         }

#### Measure all models

In [39]:
report = compare_performances(models, texts, rating, cv=10)

C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=1328, units=664, kernel_initializer="uniform")`
  if sys.path[0] == '':
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=664)`
  
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=5, kernel_initializer="uniform")`


Epoch 1/1
405/405 [==============================] - 4s 9ms/step - loss: 1.6106 - acc: 0.2568


C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=1366, units=683, kernel_initializer="uniform")`
  if sys.path[0] == '':
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:

Epoch 1/1
405/405 [==============================] - 4s 11ms/step - loss: 1.6257 - acc: 0.2494


C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=1324, units=662, kernel_initializer="uniform")`
  if sys.path[0] == '':
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:

...

C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=1336, units=668, kernel_initializer="uniform")`
  if sys.path[0] == '':
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=668)`
  
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="soft

Epoch 1/1
405/405 [==============================] - 4s 10ms/step - loss: 1.6136 - acc: 0.2593


C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=1330, units=665, kernel_initializer="uniform")`
  if sys.path[0] == '':
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=665)`
  
C:\Users\PS15S\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="soft

Epoch 1/1
405/405 [==============================] - 5s 11ms/step - loss: 1.6048 - acc: 0.2049


C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\PS15S\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


#### Performance Report

In [40]:
report_performance(report)

Model:  Baseline
Max:  0.2222222222222222
Min:  0.15555555555555556
Avg:  0.20296296296296296

Model:  HybridANN-NB
Max:  0.4222222222222222
Min:  0.32592592592592595
Avg:  0.37037037037037035

Model:  ANN
Max:  0.4
Min:  0.28888888888888886
Avg:  0.3562962962962963

Model:  Naive Bayes
Max:  0.4148148148148148
Min:  0.28888888888888886
Avg:  0.35333333333333333

Model:  Logistic Regression
Max:  0.4222222222222222
Min:  0.362962962962963
Avg:  0.39481481481481484

